In [86]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import keras
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input,InputLayer, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras import applications, initializers, Model, optimizers, metrics
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler, LabelEncoder
from tensorflow.keras.utils import to_categorical

import scipy.stats as stats

In [87]:
data = pd.read_csv("datosnuevos.csv",encoding='latin-1',index_col=0,sep = ";")
data = data.dropna()

data = data.rename(columns={'P47,':'P47'})
data['P47'] = data['P47'].str.replace(",","")

data = data.drop(['Perfil','Periodo','Curso'], axis=1)

In [88]:
data.head(2)

,Region,Ciudad,Edad,Genero,Zona,CEAD,Programa,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,P13,P14,P15,P16,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31,P32,P33,P34,P35,P36,P37,P38,P39,P40,P41,P42,P43,P44,P45,P46,P47
No,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,Cundinamarca,FUSAGASUGA,17.0000,Femenino,Zona Centro Bogotá Cundinamarca,Fusagasugá,AGRONOMIA,Totalmente de acuerdo,Ni de deacuerdo ni en desacuerdo,Medianamente de acuerdo,Levemente en desacuerdo,Totalmente en desacuerdo,Levemente en desacuerdo,Totalmente de acuerdo,Ni de deacuerdo ni en desacuerdo,Levemente en desacuerdo,Ni de deacuerdo ni en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Levemente en desacuerdo,Levemente en desacuerdo,Medianamente de acuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Levemente en desacuerdo,Levemente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Levemente en desacuerdo,Ni de deacuerdo ni en desacuerdo,Levemente en desacuerdo,Medianamente de acuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Ni de deacuerdo ni en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Levemente en desacuerdo,Levemente en desacuerdo,Levemente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Ni de deacuerdo ni en desacuerdo,Totalmente en desacuerdo
2,Bogotá,BOGOTA D.C.,18.0000,Masculino,Zona Centro Bogotá Cundinamarca,José Acevedo y Gomez,ADMINISTRACION DE EMPRESAS,Totalmente de acuerdo,Levemente en desacuerdo,Medianamente de acuerdo,Medianamente de acuerdo,Medianamente de acuerdo,Ni de deacuerdo ni en desacuerdo,Levemente en desacuerdo,Levemente en desacuerdo,Ni de deacuerdo ni en desacuerdo,Ni de deacuerdo ni en desacuerdo,Levemente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Levemente en desacuerdo,Ni de deacuerdo ni en desacuerdo,Levemente en desacuerdo,Ni de deacuerdo ni en desacuerdo,Levemente en desacuerdo,Levemente en desacuerdo,Ni de deacuerdo ni en desacuerdo,Levemente en desacuerdo,Levemente en desacuerdo,Ni de deacuerdo ni en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Medianamente de acuerdo,Totalmente en desacuerdo,Ni de deacuerdo ni en desacuerdo,Ni de deacuerdo ni en desacuerdo,Levemente en desacuerdo,Levemente en desacuerdo,Ni de deacuerdo ni en desacuerdo,Ni de deacuerdo ni en desacuerdo,Ni de deacuerdo ni en desacuerdo,Ni de deacuerdo ni en desacuerdo,Levemente en desacuerdo,Ni de deacuerdo ni en desacuerdo,Levemente en desacuerdo,Medianamente de acuerdo,Levemente en desacuerdo,Levemente en desacuerdo,Levemente en desacuerdo,Levemente en desacuerdo,Totalmente en desacuerdo,Levemente en desacuerdo


In [89]:
encode = {"Totalmente en desacuerdo":0,"Levemente en desacuerdo":1,"Ni de deacuerdo ni en desacuerdo":2,"Medianamente de acuerdo":3,"Totalmente de acuerdo":4}

In [90]:
preguntas = []
for i in range(1,48):
    preguntas.append("P"+str(i))
    data["P"+str(i)] = data["P"+str(i)].map(encode)

In [91]:
pd.set_option("display.max_columns", None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
data[preguntas].describe(include = 'all')

,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,P13,P14,P15,P16,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31,P32,P33,P34,P35,P36,P37,P38,P39,P40,P41,P42,P43,P44,P45,P46,P47
count,2931.000,2931.000,2931.000,2931.000,2931.000,2931.000,2931.000,2931.000,2931.000,2931.000,2931.000,2931.000,2931.000,2931.000,2931.000,2931.000,2931.000,2931.000,2931.000,2931.000,2931.000,2931.000,2931.000,2931.000,2931.000,2931.000,2931.000,2931.000,2931.000,2931.000,2931.000,2931.000,2931.000,2931.000,2931.000,2931.000,2931.000,2931.000,2931.000,2931.000,2931.000,2931.000,2931.000,2931.000,2931.000,2931.000,2931.000
mean,3.211,2.439,2.334,2.255,1.729,1.389,1.263,1.281,1.367,1.642,1.491,0.944,1.007,1.226,1.497,1.041,0.905,1.096,1.390,0.873,1.234,0.907,0.915,0.645,0.480,0.650,0.389,1.455,2.045,0.937,1.074,0.973,0.817,0.936,1.481,0.804,0.798,0.717,0.788,0.692,1.043,0.650,0.578,0.552,0.641,0.889,0.603
std,1.304,1.444,1.438,1.406,1.467,1.361,1.425,1.281,1.339,1.388,1.410,1.211,1.260,1.319,1.391,1.247,1.180,1.298,1.201,1.160,1.316,1.228,1.184,1.079,0.979,1.048,0.923,1.309,1.413,1.195,1.257,1.187,1.147,1.229,1.341,1.148,1.134,1.077,1.174,1.084,1.246,1.068,1.025,0.993,1.055,1.341,1.076
min,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,3.000,1.000,1.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
50%,4.000,3.000,3.000,3.000,2.000,1.000,1.000,1.000,1.000,2.000,1.000,0.000,0.000,1.000,1.000,1.000,0.000,1.000,1.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,2.000,0.000,1.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
75%,4.000,4.000,4.000,3.000,3.000,3.000,2.000,2.000,2.000,3.000,3.000,2.000,2.000,2.000,3.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,1.000,0.000,1.000,0.000,2.000,3.000,2.000,2.000,2.000,2.000,2.000,3.000,2.000,2.000,1.000,1.000,1.000,2.000,1.000,1.000,1.000,1.000,2.000,1.000
max,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000


In [92]:
pd.set_option('display.float_format', lambda x: '%.4f' % x)
data[preguntas].apply(pd.value_counts).sum(axis = 1) * 100 /137757

0   49.0632
1   15.1179
2   16.2300
3   10.8793
4    8.7095
dtype: float64

In [93]:
data["Genero"] = data["Genero"].map({"Femenino": 0, "Masculino":1})

In [94]:
data.Genero.value_counts()/len(data)

1   0.5295
0   0.4705
Name: Genero, dtype: float64

In [95]:
data.Edad = pd.qcut(data.Edad, q=4, precision = 0,labels=False)

In [96]:
data.Edad.value_counts()*100/len(data)

0   28.3180
2   27.0897
3   22.8932
1   21.6991
Name: Edad, dtype: float64

In [97]:
ramas = {"ADMINISTRACION DE EMPRESAS": "CCSS", "AGRONOMIA":"C","TECNOLOGIA EN REGENCIA DE FARMACIA":"SALUD","ZOOTECNIA":"C","COMUNICACION SOCIAL":"AYH",
"LICENCIATURA EN MATEMATICAS":"C","TECNOLOGIA EN REGENCIA DE FARMACIA (RESOLUCION 08200)":"SALUD","LICENCIATURA EN INGLES COMO LENGUA EXTRANJERA":"AYH",
"TECNOLOGIA EN SISTEMAS AGROFORESTALES":"C","LICENCIATURA EN ETNOEDUCACION":"AYH","TECNOLOGIA AGROFORESTAL":"C","TECNOLOG?A EN DESAROLLO DE SOFTWARE":"ING",
"ADMINISTRACION EN SALUD":"SALUD","TECNOLOGIA EN PRODUCCION AGRICOLA":"C","LICENCIATURA EN FILOSOFIA":"AYH","CURSOS LIBRES":"OTROS",
"TECNOLOGIA EN SISTEMAS DE COMUNICACIONES INAL?MBRICAS":"ING","TECNOLOGIA EN PRODUCCION DE AUDIO":"ING","TECNOLOGIA EN SANEAMIENTO AMBIENTAL":"C",
"TECNOLOGIA EN AUDIO":"ING","LICENCIATURA EN LENGUAS EXTRANJERAS CON ?NFASIS EN INGL?S":"AYH","PSICOLOGIA":"AYH","TECNOLOGIA INDUSTRIAL":"ING",
"ADULTO MAYOR, ACTOR SOCIAL":"OTROS","ARTES VISUALES":"AYH","TECNOLOGIA DE TELECOMUNICACIONES":"ING","TECNOLOGIA EN RADIOLOGIA E IMAGENES DIAGNOSTICAS":"SALUD",
"Música":"AYH","LICENCIATURA EN ETNOEDUCACI?N (RESOLUCI?N 26750)":"AYH","CURRICULA Y PEDAGOGIA":"CCSS","FILOSOFIA":"AYH","QUIMICA":"C",
"TECNOLOGIA EN GESTION COMERCIAL Y DE NEGOCIOS":"CCSS","Bachillerato a distancia":"OTROS","TECNOLOGIA EN SISTEMAS":"ING","TECNOLOGIA EN GESTION AGROPECUARIA":"C"}

In [98]:
data["Programa"] = data["Programa"].replace(ramas)
data.loc[data['Programa'].str.contains('INGENIER'), 'Programa'] = 'ING'
data.loc[data['Programa'].str.contains('CICLO'), 'Programa'] = 'OTROS'

In [99]:
data.Programa.value_counts()*100/len(data)

CCSS    28.2497
C       25.0426
ING     24.5650
SALUD   12.4872
AYH      8.9730
OTROS    0.6824
Name: Programa, dtype: float64

In [100]:
data['PS'] = data.loc[:,preguntas].sum(axis=1)

In [101]:
data.groupby('Genero').agg(['mean','std'])['PS']

C:\Users\JoseA\AppData\Local\Temp\ipykernel_38608\2084059586.py:1: FutureWarning: ['Region', 'Ciudad', 'Zona', 'CEAD', 'Programa'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  data.groupby('Genero').agg(['mean','std'])['PS']


,mean,std
Genero,,
0,50.4808,31.1573
1,57.2693,35.2965


In [102]:
data.groupby('Edad').agg(['mean','std'])['PS']

C:\Users\JoseA\AppData\Local\Temp\ipykernel_38608\2214983605.py:1: FutureWarning: ['Region', 'Ciudad', 'Zona', 'CEAD', 'Programa'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  data.groupby('Edad').agg(['mean','std'])['PS']


,mean,std
Edad,,
0,61.8916,33.0939
1,54.0283,31.9820
2,50.3942,31.9515
3,48.8077,35.7560


In [103]:
data.groupby('Programa').agg(['mean','std'])['PS']

C:\Users\JoseA\AppData\Local\Temp\ipykernel_38608\4178060186.py:1: FutureWarning: ['Region', 'Ciudad', 'Zona', 'CEAD'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  data.groupby('Programa').agg(['mean','std'])['PS']


,mean,std
Programa,,
AYH,53.6464,28.0381
C,54.8910,33.3939
CCSS,52.9469,34.5468
ING,55.7208,34.9944
OTROS,57.8000,24.2804
SALUD,51.8607,32.9382


In [104]:
data.Programa = data.Programa.map({"CCSS":0,"C":1,"ING":2,"SALUD":3,"AYH":4,"OTROS":5})

In [105]:
ciudades = data.Ciudad.value_counts().to_dict()
for key, val in ciudades.items():
      
    # checking for required value
    if val >= 50:
        ciudades[key] = 0
    else:
        ciudades[key] = 1
data.Ciudad = data.Ciudad.map(ciudades)

In [106]:
data.Ciudad.value_counts()

1    2027
0     904
Name: Ciudad, dtype: int64

In [107]:
data.describe()

,Ciudad,Edad,Genero,Programa,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,P13,P14,P15,P16,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31,P32,P33,P34,P35,P36,P37,P38,P39,P40,P41,P42,P43,P44,P45,P46,P47,PS
count,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000,2931.0000
mean,0.6916,1.4456,0.5295,1.5094,3.2105,2.4394,2.3337,2.2552,1.7288,1.3886,1.2634,1.2811,1.3671,1.6421,1.4906,0.9444,1.0072,1.2255,1.4971,1.0413,0.9055,1.0962,1.3903,0.8734,1.2337,0.9065,0.9150,0.6448,0.4804,0.6503,0.3893,1.4555,2.0450,0.9369,1.0737,0.9727,0.8168,0.9355,1.4814,0.8042,0.7984,0.7172,0.7878,0.6919,1.0433,0.6496,0.5783,0.5524,0.6414,0.8895,0.6025,54.0754
std,0.4619,1.1277,0.4992,1.2983,1.3039,1.4441,1.4383,1.4064,1.4672,1.3614,1.4251,1.2808,1.3395,1.3879,1.4097,1.2114,1.2602,1.3185,1.3907,1.2467,1.1802,1.2975,1.2012,1.1602,1.3156,1.2280,1.1843,1.0793,0.9789,1.0478,0.9231,1.3093,1.4125,1.1949,1.2573,1.1872,1.1473,1.2286,1.3406,1.1477,1.1343,1.0770,1.1739,1.0837,1.2464,1.0675,1.0247,0.9933,1.0555,1.3406,1.0759,33.5788
min,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
25%,0.0000,0.0000,0.0000,0.0000,3.0000,1.0000,1.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,29.0000
50%,1.0000,1.0000,1.0000,1.0000,4.0000,3.0000,3.0000,3.0000,2.0000,1.0000,1.0000,1.0000,1.0000,2.0000,1.0000,0.0000,0.0000,1.0000,1.0000,1.0000,0.0000,1.0000,1.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,2.0000,0.0000,1.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,47.0000
75%,1.0000,2.0000,1.0000,2.0000,4.0000,4.0000,4.0000,3.0000,3.0000,3.0000,2.0000,2.0000,2.0000,3.0000,3.0000,2.0000,2.0000,2.0000,3.0000,2.0000,2.0000,2.0000,2.0000,2.0000,2.0000,2.0000,2.0000,1.0000,0.0000,1.0000,0.0000,2.0000,3.0000,2.0000,2.0000,2.0000,2.0000,2.0000,3.0000,2.0000,2.0000,1.0000,1.0000,1.0000,2.0000,1.0000,1.0000,1.0000,1.0000,2.0000,1.0000,72.0000
max,1.0000,3.0000,1.0000,5.0000,4.0000,4.0000,4.0000,4.0000,4.0000,4.0000,4.0000,4.0000,4.0000,4.0000,4.0000,4.0000,4.0000,4.0000,4.0000,4.0000,4.0000,4.0000,4.0000,4.0000,4.0000,4.0000,4.0000,4.0000,4.0000,4.0000,4.0000,4.0000,4.0000,4.0000,4.0000,4.0000,4.0000,4.0000,4.0000,4.0000,4.0000,4.0000,4.0000,4.0000,4.0000,4.0000,4.0000,4.0000,4.0000,4.0000,4.0000,188.0000


In [108]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

model = ols('PS ~ Genero', data = data).fit()

anova_result = sm.stats.anova_lm(model, typ = 2)
print(anova_result)

               sum_sq        df       F  PR(>F)
Genero     33650.6755    1.0000 30.1412  0.0000
Residual 3270035.6609 2929.0000     NaN     NaN


In [109]:
groupf = data[data.Genero == 0]
groupm = data[data.Genero == 1]

stats.ttest_ind(groupf.PS, groupm.PS)

Ttest_indResult(statistic=-5.490101011612696, pvalue=4.3606096635129634e-08)

In [113]:
group0 = data[data.Edad == 0]
group1 = data[data.Edad == 1]
group2 = data[data.Edad == 2]
group3 = data[data.Edad == 3]

stats.kruskal(group0.PS, group1.PS, group2.PS, group3.PS)

KruskalResult(statistic=105.97117296785325, pvalue=8.0765863702846e-23)

In [111]:
group0 = data[data.Programa == 0]
group1 = data[data.Programa == 1]
group2 = data[data.Programa == 2]
group3 = data[data.Programa == 3]
group4 = data[data.Programa == 4]
group5 = data[data.Programa == 5]

stats.kruskal(group0.PS, group1.PS, group2.PS, group3.PS, group4.PS, group5.PS)
#stats.f_oneway(group0.PS, group1.PS)

KruskalResult(statistic=7.287796847881535, pvalue=0.20010118444332736)